# Práctica Métodos de Simulación - Parte 1

## Elena Rivas, Teresa Grau, Ignacio Casso

**Enunciado. Simulación de sucesos discretos.**

Un taller de fabricación se dedica a procesar tres tipos de piezas, para ello el
taller consta de cuatro células de procesamiento.

En el interior de cada célula se dispone de una máquina de procesado, excepto
en la célula 3 formada por dos máquinas con las mismas características, y de un
almacén (de capacidad ilimitada).

La secuencia de fabricación de cada una de las piezas así como los tiempos de
procesado (expresados en minutos y distribuidos según una triangular) en cada
célula se muestran en la siguiente tabla:


**TODO:** Escribir tabla usando sintáxis de hml:

| A | B | C |
| --- | --- | --- |
| a | b | c |

Los tiempos entre llegadas de las piezas al taller tiene carácter aleatorio. En el
fichero llegadas.txt se proporciona una muestra de tiempos entre llegadas de
piezas. Contrástese si la distribución de dichos tiempos es normal (truncada),
weibull o exponencial y estímense los parámetros de la distribución
correspondiente.

El fichero piezas.text incluye un histórico de piezas que han llegado al taller, que
nos permitirá identificar la proporción de piezas que hay de cada uno de los tres
tipos.Los tiempos de transporte de cada pieza entre las diferentes células es de 2
minutos.

A. Suponiendo que el taller trabaja de forma ininterrumpida (hay tres turnos de
trabajadores), simular el comportamiento del sistema durante 2 meses para
estimar el tiempo mínimo, medio y máximo que tardan en fabricarse los tres
tipos de piezas, el número medio de piezas esperando en cada una de las 4
células y la proporción de tiempo que están ociosas las máquinas de
procesado de las células.

B. Calcular las medidas anteriores si el tiempo de transporte de las piezas entre
las distintas células se reduce la mitad.

En caso de que fuese posible disponer de una máquina de procesado adicional,
¿en qué célula sería más beneficioso ponerla?

## Distribución de los tiempos de llegada

Tenemos que comprobar si los tiempos de llegada dados en el fichero llegadas.txt siguen una distribución normal truncada, weibull o exponencial, y estimar los parámetros correspondientes. Para ello podemos usar los contrastes de hipótesis vistos en ...

## Simulación del Sistema

A continuación simulamos el sistema...

Asumimos FIFO para cada celula de procesamiento...

Usamos los tiempos de llegada y las piezas de los ficheros (aunque tambien podriamos generarlos aleatoriamente, conociendo su distribucion)...
**TODO:** Usar un generador aleatorio? Implementarlo nosotros?

Usamos numpy.random.triangular para generar tiempos de procesado aleatorio...
**TODO:** Implementarlo nosotros?

En primer lugar escribimos los generadores de tiempos y piezas...

In [30]:
import numpy as np
import random
import queue

In [ ]:
class System():
    
    def __init__(self, transportTime, numMachines):
        self.eventsQueue = queue.PriorityQueue() # TODO: if two events occur at the exact same time,
                                                              # the priority queue will crash
        arrivals = open("E1.llegadas.txt","r")
        pieces = open("E1.piezas.txt","r")
        # open files
        
        self.transportTime = transportTime
        self.numMachines = numMachines        
        
        self.steps = [[1,2,3,4],
                      [1,2,4,2,3],
                      [2,1,3]]
        
        processing_times =[[(6,9,10), (5,8,10), (15,20,25), (8,12,16)],
                           [(1,13,15), (4,6,8), (15,18,21), (6,9,12), (27,30,39)], # El 30 es un 3 en el enunciado
                           [(7,9,11), (7,10,13), (18,23,28)]]
        
        machines = 
    # destructor
        # close files

    def simul_main(self,simulationTime):
    
        self.time = 0
    
        nextPiece()
    
        while not eventsQueue.empty():
            self.time, nextEvent = eventsQueue.get()
            nextEvent()
        
    def nextPiece(self):
        if self.time < 86400: # 2 months
            nextPieceType = self.nextPieceType()
            nextArrival = self.timeTillNextArrival()
            nextArrivalTime = self.time + nextArrival
            nextPiece = Piece(nextPieceType,nextArrivalTime)
            #eventsQueue.put(self.advancePiece(nextPiece), nextArrivalTime)
            #eventsQueue.put(self.nextPiece(), nextArrivalTime)
            # The prority queue as it is breaks if two events happen at the same time
            eventsQueue.put(self.arrivePiece(nextPiece), nextArrivalTime)
    
    def arrivePiece(self,piece):
        self.nextPiece()
        self.advance(Piece)
        
    def nextPieceType(self):
        return random.randint(1,3)
        # TODO: read from file or use true distribution
        
    def timeTillNextArrival(self):
        # arrival = arrivals.readline()
        return 10
        # TODO: read from file or use true distribution
    
    def advancePiece(self,piece):
        pieceType = piece.pieceType()
        step = piece.step()+1
        piece.step = step
        if self.steps[pieceType][step:] = []
            self.finishPiece(piece)
        else:
            next_cell = self.steps[pieceType][step]
            self.enter(nextCell,piece)
    
    def finishPiece(self,piece):
        
        pieceType = piece.pieceType
        
        totalTime = self.time - piece.arrivalTime
        
        if self.totalTime < self.minTime[pieceType]: # use dictionaries, since index needs shifting
            self.minTime[pieceType] = self.totalTime
       
        if self.totalTime > self.maxTime[pieceType]:
            self.maxTime[pieceType] = self.totalTime
        
        self.sumTimes[pieceType] += self.totalTime
        
        self.numPieces[pieceType] += 1
        
    
    def enter(self,cell,piece):
        
        machines = machines[cell]
        
        for machine in machines:
            if not machine.busy:
                self.process(machine,piece)
                return
        
        cell.queueSizeOverTime += queue.size*(self.time-lastUpdate)
        lastUpdate = time
        queue.put(piece)
        
  
    def process(self,machine,piece):
        
        machine.idleTime += (self.time - machine.idleSince)
        
        pieceType = piece.pieceType
        step = piece.step
        
        low, mode, upp = self.processingTimes[pieceType][step]
        
        processing_time = numpy.random.triangular(low,mode,up)
        
        machine.busy = True
        
        self.eventsQueue.put((time+processing_time, self.done(machine,piece)))
        
    def done(self,cell,machine,piece):
        
        machine.busy = False
        
        self.idleSince = self.time
        
        cell.exit(piece)
        
        if not cell.queue.empty():
            
            cell.queueSizeOverTime += cell.queue.qsize*(self.time-cell.lastUpdate)
            cell.lastUpdate = self.time
            piece = cell.queue.get()
            self.process(machine,piece)
        
        eventsQueue.put((self.time+self.transportTime, self.advancePiece()))
    
    # processing time (piece)

In [ ]:
class Piece:
    
    def __init__(self,pieceType, arrivalTime):
        
        self.arrivalTime = arrivalTime
        self.pieceType = pieceType
        self.step = 0

# TODO: are there structs in python?
# For our purposes, tuples would be enough, even if they are not mutable

In [ ]:
class Cell:
    
    def __init__(self):
        self.queue = queue.Queue()
        self.queueSizeOverTime = 0
        self.lastUpdate = 0

In [ ]:
class Machine:
    
    def __init__(self):
        self.busy = False
        self.idleSince = 0
        self.idleTime = 0

### Apartado A: Tiempo de transporte = 2 min.

In [ ]:
system = System(2,[1,1,2,1])
system.simulate()
system.showStats()

### Apartado B: Tiempo de transporte = 1 min.

In [ ]:
system = System(1,[1,1,2,1])
system.simulate()
system.showStats()

## Máquina de procesado adicional

Habrá que definir beneficio. Consideramos solo el tiempo?

In [ ]:
system = System(2,[2,1,2,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,2,2,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,1,3,1])
system.simulate()
system.showStats()

In [ ]:
system = System(2,[1,1,2,2])
system.simulate()
system.showStats()

In [34]:
a = [1,2,3,4]
a[4:]

[]